In [ ]:
!pip install sacremoses

In [ ]:
def test_model(dataloader_test):

    model.eval()
    
    predictions = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'labels':         batch[2],
                }  
        with torch.no_grad():        
            outputs = model(**inputs)
            
        logits = outputs[1]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    
    predictions = np.concatenate(predictions, axis=0)
    return predictions


In [ ]:
def detection(df):
    
    encoded_test_val = tokenizer.batch_encode_plus(
     df.message.values, 
     add_special_tokens=True, 
     return_attention_mask=True, 
     pad_to_max_length=True, 
     max_length=512, 
     return_tensors='pt'
    )

    input_ids_test = encoded_test_val['input_ids']
    attention_masks_test = encoded_test_val['attention_mask']
    labels_test = torch.tensor(df.label.values)

    dataset_test = TensorDataset(input_ids_test, attention_masks_test,labels_test)
    batch_size = 8
    dataloader_test = DataLoader(dataset_test, 
                                    sampler=SequentialSampler(dataset_test), 
                                    batch_size=batch_size)


    pred_test = test_model(dataloader_test) 
    preds_flat_test = np.argmax(pred_test, axis=1).flatten()
    return preds_flat_test
#     #print(preds_flat_test)

In [ ]:
import logging
import warnings
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import torch
import pandas as pd
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import MarianMTModel, MarianTokenizer
import warnings
warnings.filterwarnings('ignore')


# Set the logging level for the transformers module
logging.getLogger("transformers").setLevel(logging.ERROR)
# Suppress specific FutureWarning messages
warnings.filterwarnings("ignore", category=FutureWarning, message="The `pad_to_max_length` argument is deprecated*")

# Initialize an empty list to store comments
comments = []

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)
label_dict = {0: 0, 1: 1}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                          num_labels=len(label_dict),
                                                          output_attentions=False,
                                                          output_hidden_states=False)

model.to(device)
model.load_state_dict(torch.load('MyModel_BERT_epoch_4.model', map_location=torch.device('cpu')))


# Load the fine-tuned MarianMT model
fine_tuned_marian_model = MarianMTModel.from_pretrained("marian_output")
tokenizer_Marian = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ROMANCE")

input_csv_file = "/home/mdr614/Toxic_GRU/Fresh/random2000_for_sentimentAnalysis.csv"  # Replace with your CSV file path
comments_df = pd.read_csv(input_csv_file, header=None, names=["comment"])

# Create an empty list to store comments and their associated outputs
comments_with_outputs = []

count=0
for inp_sentence in comments_df["comment"]:
    # Collect comment from the user or exit the loop
    comment = inp_sentence

    # Add the comment to the list
    comments.append(comment)

    # Create a pandas DataFrame from the collected comments
    data = {'message': comments, 'label': 0}
    df = pd.DataFrame(data)

    #for MarianMT
    uncivil_sentence = comment
    inputs = tokenizer_Marian(uncivil_sentence, return_tensors="pt", max_length=1024, trancation=True)
    generated_ids = fine_tuned_marian_model.generate(**inputs, max_new_tokens=1024)
    generated_civil_sentence = tokenizer_Marian.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
    subword_units = generated_civil_sentence.split('▁')
    # Remove empty strings resulting from the split
    subword_units = [unit for unit in subword_units if unit]
    # Join subword units into words
    cleaned_sentence = ' '.join(subword_units)

                    # Add the comment to the list
    comments.append(cleaned_sentence)
    # Create a pandas DataFrame from the collected comments
    data = {'message': comments, 'label': 0}
    df = pd.DataFrame(data)

    preds_flat_test = detection(df)

    if preds_flat_test[-1]==0:
        print("\033[92mGenerated civil alternative-MarianMT:\033[0m", cleaned_sentence)
        comments_with_outputs.append((comment, cleaned_sentence))
        print("\n")
    else:
        print("\033[91mGenerated civil alternative-MarianMT:\033[0m", cleaned_sentence)
        comments_with_outputs.append((comment, "Not Generated"))
        print("\n")

    count+=1
    print("count:", count)

output_df = pd.DataFrame(comments_with_outputs, columns=["uncivil", "civil"])
output_csv_file = "/content/drive/MyDrive/civilAlternatives_by_MarianMT_random2000.csv"
output_df.to_csv(output_csv_file, index=False)
